## Inputs
---

In [1]:
# reading data and working with arrays
import h5py, nrrd
import numpy as np

# data paths
data_path = '/groups/scicompsoft/home/fleishmang/rnr-exm-work/test_data/c_elegan_pair6.h5'
hdf5_file = h5py.File(data_path, 'r')
fix = hdf5_file['/fixed'][...]
mov = hdf5_file['/move'][...]

# assumed spacings (there are no provided spacings, this is a problem)
fix_spacing = np.array([0.4, 0.1625, 0.1625])
mov_spacing = np.array([0.4, 0.1625, 0.1625])

# # write some channels
# nrrd.write('./fix.nrrd', fix.transpose(2,1,0), compression_level=2)
# nrrd.write('./mov.nrrd', mov.transpose(2,1,0), compression_level=2)

## Masking
---

### fixed

In [2]:
# tools for coarse whole brain segmentation
from bigstream import level_set
from scipy.ndimage import zoom, binary_closing, binary_dilation

# # get small mask
# fix_skip = fix[::4, ::4, ::4]
# fix_mask = np.zeros(fix_skip.shape, dtype=np.uint8)
# skip_spacing = fix_spacing * [4, 4, 4]
# fix_mask_ = level_set.foreground_segmentation(
#     fix_skip[60:, :, 200:], skip_spacing,
#     mask_smoothing=2,
#     iterations=[80,40,10],
#     smooth_sigmas=[12,6,3],
#     lambda2=1.0,
# )
# fix_mask[60:, :, 200:] = fix_mask_

# # smooth and enlarge mask
# fix_mask = binary_closing(fix_mask, np.ones((2,2,2))).astype(np.uint8)
# fix_mask = binary_dilation(fix_mask, np.ones((2,2,2))).astype(np.uint8)
# fix_mask = zoom(fix_mask, np.array(fix.shape) / fix_skip.shape, order=0)

# # write result
# nrrd.write('./fix_mask.nrrd', fix_mask.transpose(2,1,0), compression_level=2)

# load precomputed mask
fix_mask, _ = nrrd.read('./fix_mask.nrrd')
fix_mask = fix_mask.transpose(2,1,0)

### moving

In [3]:
# tools for coarse whole brain segmentation
from bigstream import level_set
from scipy.ndimage import zoom, binary_closing, binary_dilation

# # get small mask
# mov_skip = mov[::4, ::8, ::8]
# mov_mask = np.zeros(mov_skip.shape, dtype=np.uint8)
# skip_spacing = mov_spacing * [4, 8, 8]
# mov_mask_ = level_set.foreground_segmentation(
#     mov_skip[:, :, 90:], skip_spacing,
#     mask_smoothing=2,
#     iterations=[80,40,10],
#     smooth_sigmas=[24,12,6],
#     lambda2=32.0,
# )
# mov_mask[:, :, 90:] = mov_mask_

# # smooth and enlarge mask
# mov_mask = binary_closing(mov_mask, np.ones((2,2,2))).astype(np.uint8)
# mov_mask = binary_dilation(mov_mask, np.ones((2,2,2))).astype(np.uint8)
# mov_mask = zoom(mov_mask, np.array(mov.shape) / mov_skip.shape, order=0)

# # write result
# nrrd.write('./mov_mask.nrrd', mov_mask.transpose(2,1,0), compression_level=2)

# load precomputed mask
mov_mask, _ = nrrd.read('./mov_mask.nrrd')
mov_mask = mov_mask.transpose(2,1,0)

## Alignment
---

### affine

In [4]:
# alignment functions
from bigstream.align import alignment_pipeline
from bigstream.transform import apply_transform

# define alignment steps
common_kwargs = {
    'alignment_spacing':2.0,
    'shrink_factors':(2,1),
    'smooth_sigmas':(12.,2.,),
    'optimizer_args':{
        'learningRate':0.25,
        'minStep':0.,
        'numberOfIterations':6000,
    },
}

rigid_kwargs = {
    'initial_condition':'CENTER',
    'fix_mask':fix_mask,
    'mov_mask':mov_mask,
}

affine_kwargs = {
    'fix_mask':fix_mask,
    'mov_mask':mov_mask,
    'smooth_sigmas':(2.,2.,),
}

deform_kwargs = {
    'alignment_spacing':1.0,
    'shrink_factors':(1,),
    'smooth_sigmas':(1.,),
    'control_point_spacing':100.0,
    'control_point_levels':(1,),
    'optimizer_args':{
        'learningRate':0.01,
        'minStep':0.002,
        'numberOfIterations':80,
    },
}

steps = [('rigid',  {**common_kwargs, **rigid_kwargs},),
         ('affine', {**common_kwargs, **affine_kwargs},),]
         # ('deform', {**common_kwargs, **deform_kwargs},),]

IBM Spectrum LSF 10.1.0.0 build 601088, Apr 15 2022
Copyright International Business Machines Corp. 1992, 2016.
US Government Users Restricted Rights - Use, duplication or disclosure restricted by GSA ADP Schedule Contract with IBM Corp.

  binary type: linux3.10-glibc2.17-x86_64


In [5]:
%%time

# run alignment
affine = alignment_pipeline(
    fix, mov,
    fix_spacing, mov_spacing,
    steps,
    # fix_mask=fix_mask,
    # mov_mask=mov_mask,
)

LEVEL:  0  ITERATION:  0  METRIC:  -0.44082702054810075
LEVEL:  0  ITERATION:  1  METRIC:  -0.45295893627183126
LEVEL:  0  ITERATION:  2  METRIC:  -0.46545636598348106
LEVEL:  0  ITERATION:  3  METRIC:  -0.4783421879490885
LEVEL:  0  ITERATION:  4  METRIC:  -0.4882917533298286
LEVEL:  0  ITERATION:  5  METRIC:  -0.48109121029041935
LEVEL:  0  ITERATION:  6  METRIC:  -0.4920724082656138
LEVEL:  0  ITERATION:  7  METRIC:  -0.5053110916926844
LEVEL:  0  ITERATION:  8  METRIC:  -0.5198090152730047
LEVEL:  0  ITERATION:  9  METRIC:  -0.531993744757517
LEVEL:  0  ITERATION:  10  METRIC:  -0.5401962859003538
LEVEL:  0  ITERATION:  11  METRIC:  -0.5472350731386676
LEVEL:  0  ITERATION:  12  METRIC:  -0.5601629469004262
LEVEL:  0  ITERATION:  13  METRIC:  -0.5744362913575204
LEVEL:  0  ITERATION:  14  METRIC:  -0.5825859992250908
LEVEL:  0  ITERATION:  15  METRIC:  -0.587175440291958
LEVEL:  0  ITERATION:  16  METRIC:  -0.5995638284128165
LEVEL:  0  ITERATION:  17  METRIC:  -0.6126153214592384


In [6]:
# apply transform and save results
affine_aligned = apply_transform(
    fix, mov,
    fix_spacing, mov_spacing,
    transform_list=[affine,],
)

# write results
np.savetxt('affine.mat', affine)
nrrd.write('./affine.nrrd', affine_aligned.transpose(2,1,0), compression_level=2)

# load precomputed results
affine = np.loadtxt('./affine.mat')

In [7]:
from bigstream.utility import matrix_to_displacement_field

# load affine, convert to displacement, convert to voxel units
affine = np.loadtxt('./affine.mat')
deform = matrix_to_displacement_field(affine, fix.shape, spacing=fix_spacing)
deform = (deform / fix_spacing).astype(np.float32)
deform = np.around(deform, decimals=3)

# save as h5 file
with h5py.File('../submission/c_elegan_test.h5', 'a') as hf:
    hf.create_dataset('pair6', data=deform, compression='gzip')